In [1]:
import sys
sys.path.append('/oak/stanford/groups/rbaltman/luyang/scripts/POPDx/code/')
%load_ext autoreload
%autoreload 2

### ICD-10 embeddings

In [2]:
from embeddings import *

In [3]:
import transformers
print(transformers.__version__)

3.2.0


In [4]:
l2i, i2l, onto_embeddings = ontology_emb(dim=500, ICD_network_file = '../data/19.csv', save_dir = './embeddings/', use_pretrain = True)

In [6]:
biboert_embeddings, biboert_embeddings_dict = run_bert(use_pretrain = False)

cuda:0
Batch # 1
Batch # 2
Batch # 3
Batch # 4
Batch # 5
Batch # 6
Batch # 7
Batch # 8
Batch # 9
Batch # 10
Batch # 11
Batch # 12
Batch # 13
Batch # 14
Batch # 15
Batch # 16
Batch # 17
Batch # 18
Batch # 19
Batch # 20
Batch # 21
Batch # 22
Batch # 23
Batch # 24
Batch # 25
Batch # 26
Batch # 27
Batch # 28
Batch # 29
Batch # 30
Batch # 31
Batch # 32
Batch # 33
Batch # 34
Batch # 35
Batch # 36
Batch # 37
Batch # 38
Batch # 39
Batch # 40
Batch # 41
Batch # 42
Batch # 43
Batch # 44
Batch # 45
Batch # 46
Batch # 47
Batch # 48
Batch # 49
Batch # 50
Batch # 51
Batch # 52
Batch # 53
Batch # 54
Batch # 55
Batch # 56
Batch # 57
Batch # 58
Batch # 59
Batch # 60
Batch # 61
Batch # 62
Batch # 63
Batch # 64
Batch # 65
Batch # 66
Batch # 67
Batch # 68
Batch # 69
Batch # 70
Batch # 71
Batch # 72
Batch # 73
Batch # 74
Batch # 75
Batch # 76
Batch # 77
Batch # 78
Batch # 79
Batch # 80
Batch # 81
Batch # 82
Batch # 83
Batch # 84
Batch # 85
Batch # 86
Batch # 87
Batch # 88
Batch # 89
Batch # 90
Batch # 91
B

In [7]:
with open('../data/mc_icd10_labels.txt','r') as f:
    labels = f.readlines()
labels = [x.strip() for x in labels] 
labels_idx = [l2i[l] for l in labels]
onto_embeddings = onto_embeddings[labels_idx, :]
print(onto_embeddings.shape,  biboert_embeddings.shape)

(12803, 500) (12803, 768)


In [8]:
Y_emb_concat = np.concatenate((onto_embeddings, biboert_embeddings), axis=1)
print(Y_emb_concat.shape)

(12803, 1268)


In [9]:
np.save('../data/icd10_label_embed.npy', Y_emb_concat)

### Phecode embeddings


In [11]:
df_phecode = pd.read_csv('../data/Phecode_map_icd10.csv')

In [12]:
df_phecode.head()

,ICD10,PHECODE,Exl. Phecodes,Excl. Phenotypes
0,A009,8.0,001-009.99,Intestinal infection
1,A010,8.5,001-009.99,Intestinal infection
2,A011,8.0,001-009.99,Intestinal infection
3,A012,8.0,001-009.99,Intestinal infection
4,A013,8.0,001-009.99,Intestinal infection


In [18]:
with open('../data/phecode_labels.txt', 'r') as f:
    phecode_labels = f.readlines() 
phecode_labels = [float(x.strip()) for x in phecode_labels] 

In [20]:
phecode_labels_emb = np.zeros((1538, 1268))
for i,phelabel in enumerate(phecode_labels):
    print(phelabel)
    corr_icd10 = df_phecode.loc[df_phecode.PHECODE==phelabel].ICD10.tolist()
    corr_idx = []
    for l in corr_icd10:
        corr_idx.append(labels.index(l))
    assert (corr_icd10 == np.array(labels)[corr_idx]).all()
    phecode_labels_emb[i,:] = Y_emb_concat[corr_idx, :].mean(axis=0)

8.0
8.5
8.51
8.52
8.6
8.7
10.0
31.0
31.1
38.0
38.1
38.2
41.0
41.1
41.2
41.21
41.4
53.0
53.1
54.0
70.0
70.1
70.2
70.3
70.4
70.9
71.1
78.0
79.0
79.1
79.2
80.0
81.0
90.0
90.2
90.3
110.1
110.11
110.12
110.13
110.2
112.0
112.3
117.0
117.1
117.2
117.3
117.4
130.0
130.1
131.0
132.0
132.1
133.0
134.0
134.1
136.0
145.0
145.1
145.2
145.3
145.4
145.5
149.0
149.1
149.2
149.3
149.4
149.9
150.0
151.0
153.0
153.2
153.3
155.0
155.1
157.0
158.0
159.0
159.2
159.3
159.4
164.0
165.0
165.1
170.1
170.2
172.11
172.2
172.3
173.0
174.1
174.11
174.3
175.0
180.1
180.3
182.0
184.0
184.1
184.11
184.2
185.0
187.0
187.1
187.2
187.8
189.0
189.11
189.12
189.2
189.21
189.4
190.0
191.0
191.1
191.11
193.0
194.0
195.0
195.1
195.3
196.0
197.0
198.0
198.1
198.2
198.3
198.4
198.5
198.6
198.7
199.0
199.4
200.0
200.1
201.0
202.0
202.2
202.21
202.23
202.24
204.0
204.1
204.11
204.12
204.2
204.21
204.22
204.3
204.4
208.0
210.0
211.0
212.0
213.0
214.0
214.1
215.0
216.0
217.0
217.1
218.1
218.2
220.0
221.0
222.0
223.0
224.0
224.1
22

770.0
771.0
771.1
772.0
772.1
773.0
780.0
781.0
781.2
782.3
782.6
783.0
785.0
788.0
789.0
790.0
790.1
790.6
791.0
793.0
793.2
794.0
797.0
797.1
798.0
798.1
800.0
800.1
800.2
800.3
800.4
801.0
802.0
803.0
803.1
803.2
803.3
804.0
805.0
807.0
809.0
817.0
818.0
819.0
830.0
835.0
836.0
840.0
840.3
850.0
851.0
853.0
854.0
857.0
858.0
859.0
860.0
870.0
870.1
870.2
870.3
870.4
870.6
870.8
871.0
871.3
871.4
872.0
874.0
907.0
913.0
915.0
916.0
930.0
931.0
938.0
938.1
938.2
939.0
941.0
942.0
946.0
947.0
949.0
949.1
952.0
958.0
958.1
958.2
960.0
960.1
960.2
961.0
961.1
962.0
962.3
963.0
963.1
964.0
964.1
965.0
965.1
965.3
966.0
967.0
969.0
971.0
972.0
973.0
974.0
975.0
976.0
979.0
981.0
983.0
985.0
986.0
987.0
988.0
989.0
990.0
1000.0
1001.0
1002.0
1004.0
1005.0
1006.0
1007.0
1008.0
1009.0
1010.0
1011.0
1012.0
1013.0
1014.0
1015.0
1019.0
1100.0


In [21]:
np.save('../data/phecode_label_embed.npy', phecode_labels_emb)